In [2]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import feather
from extras import *
pd.set_option('display.max_columns', None)

In [3]:
ja = Player('Ja Morant', '1629630', '1610612763', ['2019-20','2020-21','2021-22'], 'm/moranja01', ['2020', '2021', '2022'])
drose = Player('Derrick Rose', '201565', '1610612741', ['2008-09','2009-10','2010-11'], 'r/rosede01', ['2009', '2010', '2011'])
west = Player('Russell Westbrook', '201566', '1610612760', ['2008-09','2009-10','2010-11'], 'w/westbru01', ['2009', '2010', '2011'])
iver = Player('Allen Iverson', '947', '1610612755', ['1996-97','1997-98','1998-99'], 'i/iversal01', [ '1997', '1998', '1999'])
players = [ja, drose, west, iver]
players2 = [ja, drose]

Processing data for the total points graph

In [3]:
for player in players2:
    player_data = pd.DataFrame()
    for year in player.bballrefseasons():
        new_season = pd.read_json('../data/'+ player.nbaid() +'/'+ year +'boxscores.json')
        player_data = pd.concat([player_data, new_season], axis=0)
    player_data = player_data.reset_index(drop=True)
    player_data = player_data[['PTS']].cumsum()
    feather.write_dataframe(player_data, './Rdata/'+ player.nbaid() + '_cumpts.feather')

Processing data for radar plots

In [11]:
pd.read_json('../data/' + players[0].nbaid() + '/season_avg_per100.json')

,By Year,TEAM,GP,MIN,PTS,FGM,FGA,FG%,3PM,3PA,3P%,FTM,FTA,FT%,OREB,DREB,REB,AST,TOV,STL,BLK,PF,FP,DD2,TD3,+/-,OffRtg,DefRtg,NetRtg,AST%,AST/TO,AST Ratio,OREB%,DREB%,REB%,TO Ratio,eFG%,TS%,USG%,PACE,PIE,PTS OFF TO,2nd PTS,FBPs,PITP,OppPTS OFF TO,Opp2nd PTS,OppFBPs,OppPITP,BLKA,PFD,%FGA2PT,%FGA3PT,%PTS2PT,%PTS2PT MR,%PTS3PT,%PTSFBPs,%PTSFT,%PTSOffTO,%PTSPITP,2FGM%AST,2FGM%UAST,3FGM%AST,3FGM%UAST,FGM%AST,FGM%UAST,%FGM,%FGA,%3PM,%3PA,%FTM,%FTA,%OREB,%DREB,%REB,%AST,%TOV,%STL,%BLK,%BLKA,%PF,%PFD,%PTS
0,2021-22,MEM,57,47.2,39.1,14.5,29.4,49.3,2.2,6.4,34.4,7.9,10.4,76.1,1.9,6.2,8.1,9.6,4.9,1.6,0.5,2.1,64.9,10,1,4.7,115.3,111.2,4.1,33.0,1.96,19.8,3.7,12.4,8.0,10.1,53.0,57.5,33.0,101.46,16.3,5.6,4.1,6.2,23.7,14.7,12.0,12.8,47.1,2.4,8.2,78.2,21.8,62.9,2.3,16.9,15.9,20.2,14.3,60.6,24.8,75.2,51.1,48.9,28.8,71.2,33.3,32.0,21.2,21.7,44.3,43.0,13.5,17.8,16.5,39.3,36.9,19.0,8.9,37.5,11.9,41.1,33.9
1,2020-21,MEM,63,47.0,27.6,9.8,21.9,44.9,1.7,5.5,30.3,6.2,8.5,72.8,1.4,4.4,5.8,10.7,4.6,1.3,0.3,2.0,50.6,14,1,2.5,114.7,112.1,2.6,32.1,2.29,25.9,2.7,9.1,5.9,11.3,48.7,53.7,26.7,102.16,11.3,4.6,2.4,4.1,15.3,15.9,12.6,13.4,47.5,1.6,7.1,74.8,25.2,59.3,4.0,18.2,15.0,22.5,16.5,55.3,29.4,70.6,60.3,39.7,34.7,65.3,22.9,24.4,15.3,18.5,35.2,37.3,11.9,12.9,12.6,40.6,37.2,15.6,6.5,31.5,11.0,36.6,24.0
2,2019-20,MEM,67,46.2,26.6,10.0,20.9,47.7,1.3,4.0,33.5,5.3,6.9,77.6,1.1,4.6,5.8,10.9,4.9,1.3,0.4,2.4,50.0,15,2,-0.4,107.5,108.1,-0.6,35.4,2.22,27.4,2.4,9.0,5.9,12.3,50.9,55.6,25.6,103.79,13.0,4.1,2.1,3.8,15.9,17.0,13.1,12.3,42.6,2.0,6.1,80.9,19.1,64.9,4.9,15.1,14.2,20.0,15.3,60.0,28.2,71.8,55.0,45.0,31.8,68.2,24.5,24.4,13.6,14.1,32.6,32.0,11.4,13.0,12.7,44.4,31.7,18.8,7.9,37.7,11.9,30.0,24.7


In [17]:
data = pd.DataFrame()
for player in players2:
    player_data = pd.read_json('../data/' + player.nbaid() + '/season_avg_per100.json')
    player_data['PLAYER'] = [player.name()]*len(player_data)
    player_data['SEASON'] = range(3,0,-1)
    data = pd.concat([data, player_data[['PLAYER', 'SEASON', 'PTS', 'AST', 'REB', 'TOV', 'STL', 'OffRtg', 'DefRtg', 'TS%', 'USG%', '%PTS2PT', '%PTSPITP', '%PTSFBPs', 'FGM%UAST']]],
                    axis=0)
    
feather.write_dataframe(data,'./Rdata/stats.feather')

In [4]:
pd.read_json('../data/byseason/2021-22.json')

,PLAYER,TEAM,AGE,GP,W,L,MIN,PTS,FGM,FGA,FG%,3PM,3PA,3P%,FTM,FTA,FT%,OREB,DREB,REB,AST,TOV,STL,BLK,PF,FP,DD2,TD3,+/-,OFFRTG,DEFRTG,NETRTG,AST%,AST/TO,AST RATIO,OREB%,DREB%,REB%,TO RATIO,EFG%,TS%,USG%,PACE,PIE,POSS,%FGA2PT,%FGA3PT,%PTS2PT,%PTS2PT MR,%PTS3PT,%PTSFBPS,%PTSFT,%PTSOFFTO,%PTSPITP,2FGM%AST,2FGM%UAST,3FGM%AST,3FGM%UAST,FGM%AST,FGM%UAST,PTSOFF TO,2NDPTS,FBPS,PITP,OPP PTSOFF TO,OPP2ND PTS,OPPFBPS,OPPPITP,BLKA,PFD,%FGM,%FGA,%3PM,%3PA,%FTM,%FTA,%OREB,%DREB,%REB,%AST,%TOV,%STL,%BLK,%BLKA,%PF,%PFD,%PTS,RA FGM,RA FGA,RA FG%,ITP FGM,ITP FGA,ITP FG%,MR FGM,MR FGA,MR FG%,LC3 FGM,LC3 FGA,LC3 FG%,RC3 FGM,RC3 FGA,RC3 FG%,C3 FGM,C3 FGA,C3 FG%,AB3 FGM,AB3 FGA,AB3 FG%
0,Joel Embiid,PHI,28,68,45,23,49.5,44.8,14.4,28.8,49.9,2.0,5.4,37.1,14.1,17.3,81.4,3.1,14.0,17.2,6.1,4.6,1.7,2.1,3.9,81.3,46,2,7.9,115.6,107.8,7.9,23.6,1.33,13.0,6.8,27.3,17.6,9.8,53.4,61.6,37.5,96.96,21.2,4640,81.2,18.8,55.1,15.9,13.4,6.1,31.5,12.3,39.2,51.3,48.7,71.0,29.0,54.1,45.9,5.5,4.4,2.7,17.6,13.9,12.9,14.7,47.8,1.8,12.5,35.6,34.3,16.9,17.5,60.7,61.4,45.3,39.4,40.4,24.7,39.6,22.4,40.4,42.7,21.8,58.0,38.7,4.2,5.7,73.5,1.8,4.5,40.3,2.4,5.8,42.2,0.0,0.1,28.6,0.0,0.1,33.3,0.1,0.2,30.8,1.3,3.5,37.6
1,Giannis Antetokounmpo,MIL,27,67,45,22,46.5,42.2,14.5,26.2,55.3,1.5,5.1,29.3,11.7,16.2,72.2,2.8,13.6,16.4,8.2,4.6,1.5,1.9,4.5,79.9,46,4,8.4,115.9,107.9,8.0,30.4,1.77,17.8,6.0,26.7,16.8,10.0,58.2,63.3,34.0,103.12,21.0,4743,80.6,19.4,61.7,8.5,10.6,14.4,27.7,13.2,53.2,43.2,56.8,42.3,57.7,43.1,56.9,5.6,4.5,6.1,22.5,14.5,11.4,9.4,39.9,1.3,12.4,35.0,30.8,11.0,13.9,59.9,63.1,31.8,36.6,35.7,33.0,36.3,20.2,46.7,33.7,24.7,58.1,36.4,6.6,8.7,76.6,1.3,3.3,40.8,1.3,3.1,41.5,0.0,0.1,25.0,0.0,0.0,0.0,0.0,0.1,16.7,1.0,3.5,29.9
2,Luka Doncic,DAL,23,65,44,21,50.0,40.1,13.9,30.5,45.7,4.4,12.4,35.3,7.9,10.6,74.4,1.2,11.7,12.9,12.3,6.3,1.6,0.8,3.1,75.0,44,10,3.2,113.8,110.3,3.5,45.8,1.95,22.9,2.5,23.0,12.9,11.8,52.9,57.1,36.8,96.18,19.1,4604,59.4,40.6,47.6,8.8,32.6,4.0,19.8,11.5,38.9,10.7,89.3,21.9,78.1,14.2,85.8,4.6,2.6,1.6,15.6,14.9,14.8,13.0,50.7,1.0,8.7,34.1,34.9,30.2,30.5,44.8,45.6,12.5,33.7,29.1,52.4,48.3,23.4,18.4,29.1,16.0,41.4,35.3,2.5,3.4,73.8,3.0,6.1,49.2,1.2,3.3,37.7,0.1,0.3,33.3,0.0,0.2,20.0,0.1,0.5,29.0,3.0,8.2,35.8
3,Ja Morant,MEM,22,57,36,21,47.2,39.1,14.5,29.4,49.3,2.2,6.4,34.4,7.9,10.4,76.1,1.9,6.2,8.1,9.6,4.9,1.6,0.5,2.1,64.9,10,1,4.7,115.3,111.2,4.1,33.0,1.96,19.8,3.7,12.4,8.0,10.1,53.0,57.5,33.0,101.46,16.3,4003,78.2,21.8,62.9,2.3,16.9,15.9,20.2,14.3,60.6,24.8,75.2,51.1,48.9,28.8,71.2,5.6,4.1,6.2,23.7,14.7,12.0,12.8,47.1,2.4,8.2,33.3,32.0,21.2,21.7,44.3,43.0,13.5,17.8,16.5,39.3,36.9,19.0,8.9,37.5,11.9,41.1,33.9,5.3,7.9,66.9,3.0,7.0,42.9,0.3,1.2,26.1,0.1,0.2,46.2,0.1,0.2,38.5,0.2,0.5,42.3,1.4,4.0,33.9
4,Nikola Jokic,DEN,27,74,46,28,48.2,39.0,14.9,25.5,58.3,1.9,5.6,33.7,7.4,9.1,81.0,4.0,15.8,19.8,11.4,5.5,2.1,1.2,3.7,84.5,66,19,8.6,117.3,108.9,8.4,38.8,2.08,24.8,9.0,31.3,20.9,11.9,62.0,66.1,30.9,99.46,23.0,5136,78.0,22.0,66.6,6.8,14.5,6.4,18.9,11.1,59.8,58.0,42.0,91.8,8.2,62.3,37.7,4.3,5.5,2.5,23.3,17.6,11.4,12.1,50.7,1.3,9.4,33.6,29.5,16.3,17.2,42.8,42.4,45.3,42.9,43.3,37.4,37.2,30.9,34.8,29.9,21.8,45.5,33.3,4.7,6.6,71.9,3.4,5.5,60.9,0.9,1.7,53.5,0.0,0.1,33.3,0.0,0.1,20.0,0.0,0.1,27.3,1.3,3.7,34.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
403,Andre Iguodala,GSW,38,31,20,11,48.1,9.9,3.7,9.7,38.0,1.4,5.9,23.0,1.2,1.6,75.0,1.7,6.3,8.0,9.1,2.2,2.2,1.8,2.7,42.6,0,0,9.1,105.7,97.0,8.7,26.3,4.07,41.8,3.5,11.7,7.8,10.3,45.0,47.8,11.3,99.57,10.3,1253,38.8,61.2,46.8,3.2,41.1,11.3,12.1,18.5,43.5,62.1,37.9,94.1,5.9,73.9,26.1,1.8,0.7,1.1,4.3,13.9,13.5,11.0,40.2,0.2,2.9,9.6,11.4,10.2,14.5,7.6,7.8,18.3,

In [ ]:
def get_percentiles(df, cols):
    
for player in players2:
    for season in player.nbaseasons():
        df = pd.read_json('../data/byseason/' + season + '.json')
        

Processing data for beeswarm plots

In [5]:
def getBin(num, q25, q75):
    if num < q25:
        return 'LOW'
    elif (num >= q25) and (num < q75):
        return 'MEDIUM'
    else:
        return 'HIGH'

allyears = pd.DataFrame()

for year in players2[0].nbaseasons():
    drivespg = pd.read_json('../data/1629630/20' + year[-2:] + 'drivespergame.json')
    drivespg.describe()
    q25, q75 = drivespg['DRIVES'].quantile(.25), drivespg['DRIVES'].quantile(.75)
    drivespg['BIN'] = drivespg['DRIVES'].apply(getBin, args=(q25, q75))
    drivespg['X'] = ['']*len(drivespg)
    drivespg['YEAR'] = [year]*len(drivespg)
    allyears = pd.concat([allyears, drivespg], axis=0)

feather.write_dataframe(allyears, './Rdata/drivespg.feather')

Processing data for shot_plots

In [4]:
for player in players:
    allshots = pd.DataFrame()
    for season in player.nbaseasons():
        allshots = pd.concat([allshots, pd.read_json('../data/' + player.nbaid() + '/' + season + '_shots.json')], axis=0)
    feather.write_dataframe(allshots, './Rdata/'+ player.nbaid() +'_allshots.feather')